In [ ]:
!pip install -q google-generativeai PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.6 MB/s eta 0:00:00


In [ ]:
import os
import time
import random
import re
import PyPDF2
import traceback
import google.generativeai as genai
from google import genai as google_genai
from google.genai import types
from google.colab import drive, userdata
from pathlib import Path
from queue import Queue
from threading import Thread, Lock
import numpy as np

if not os.path.exists('/content/drive/MyDrive'):
    drive.mount('/content/drive')
else:
    print("Drive already mounted")

# Configuration
INPUT_DIR = "/content/drive/MyDrive/Schools_Books"  # Update this path
OUTPUT_DIR = "/content/drive/MyDrive/Schools_Books_Summaries"
API_KEYS = [userdata.get('GOOGLE_API_KEY'), userdata.get('GEMINI_KEY'), userdata.get('GEMINI_KEY_2'), userdata.get('GEMINI_KEY_3'), userdata.get('GEMINI_KEY_4'), userdata.get('GEMINI_KEY_5'), userdata.get('GEMINI_KEY_6'), userdata.get('GEMINI_KEY_7')]
#API_KEYS = [userdata.get('GEMINI_KEY_5'), userdata.get('GEMINI_KEY_6')]
DELAY_SECONDS = 15
MAX_RETRIES = 5
SUMMARY_VERSIONS = 3
FORCE_REGENERATE = []
SUPPORTED_LANGS = {
    "en": "English",
    "hi": "हिंदी",
    "hinglish": "Hinglish",
    "sa": "संस्कृतम्"
}
PROBLEMATIC_FILES = []
LANGUAGE_BINDINGS = {
    "hindi": ["hi", "hinglish"],
    "sanskrit": ["sa"],
    "english": ["en"],
    "regional": ["hi", "en"],  # For state board regional language books
    # Add more as needed
}

CULTURAL_EXAMPLES = {
    "north": ["Punjab farming techniques", "Kashmir water conservation",
             "Rajasthan desert adaptations"],
    "south": ["Kerala backwaters ecosystem", "Tamil Nadu temple architecture",
             "Karnataka silk production"],
    "east": ["West Bengal monsoon patterns", "Odisha tribal art materials",
            "Assam tea processing"],
    "west": ["Maharashtra coastal erosion", "Gujarat salt farming",
            "Goan food preservation"]
}

EXPERIMENT_TEMPLATE = {
    "required_elements": [
        "Household materials list 🧂🥤",
        "Safety tip ⚠️",
        "Observation prompt 👀",
        "Real-world connection 🌍"
    ],
    "sample": {
        "States of Matter": [
            "Watch sugar vanish in tea 🍵→❌",
            "Freeze soap bubbles ❄️→🔴",
            "Track wet cloth drying ☀️→⏳"
        ]
    }
}

ADVANCED_CONCEPT_MAP = {
    "primary": {
        "Plasma": "Fourth state in lightning ⚡",
        "BEC": "Super-cold science magic ❄️✨"
    },
    "middle": {
        "Nanotech": "Tiny robot doctors 🤖💊",
        "Quantum": "Particles that teleport 🌀"
    },
    "secondary": {  # Added secondary level concepts
        "CRISPR": "Gene editing scissors ✂️🧬",
        "Blockchain": "Digital trust chains 🔗💻",
        "Metamaterials": "Light-bending substances 🌈🔮"
    }
}

# Pedagogical Prompt Template
PEDAGOGICAL_PROMPT = """<NCERT_SUMMARY_SPEC version="2.5">
  <CONTEXT>
    Class: {class_num} ({subject})
    Chapter: {chapter_num}
    Language: {lang_name} ({lang_code})
    Curriculum: CBSE/{state_board}
    Cultural Context: {cultural_ref}
  </CONTEXT>

  <STRUCTURE format="markdown">
    # [Class {class_num}] {chapter_title}

    ## 🌟 Core Concepts
    {concept_style}

    ## 📘 Key Learnings
    {learning_points}

    ## 🧩 Active Learning
    - Activity: {activity}
    - Discussion: {discussion_topic}

    ## 📝 Assessment Prep
    {assessment_types}

    ## 🌏 Bharatiya Context
    {indian_examples}
  </STRUCTURE>

  <RULES>
    1. Strict NCERT alignment
    2. Bloom's: {blooms_level}
    3. Language: {lang_rules}
    4. Quality: {quality_checks}
    5. Token limit: {token_limit}
  </RULES>
</NCERT_SUMMARY_SPEC>

Text Content:
{text}"""

# Child-Friendly Prompt Template
CHILD_FRIENDLY_PROMPT_V2 = """📚 Create a Class {class_num} summary for '{chapter_title}' in {lang_name} that:
1. Explains concepts like telling a story to {age_group}-year-olds 🧒👧
2. Uses 3 real-life experiments kids can try at home 🥼
3. Includes cultural examples from 3 different Indian states 🇮🇳
4. Adds 1 "Wow Fact" about advanced science (keep it simple!) 💫

Follow this format:
# {chapter_title}
✨ **Big Ideas** (3-5 emoji-powered points)
   - Start with "Imagine..." or "Did you know..."
   - Add comparison: "Like when..."

🔍 **Explore Through Play**
   - Experiment 1: (Materials: household items)
   - Experiment 2: (Include safety tip ⚠️)
   - Experiment 3: (Measure & observe 🔍)

🌍 **India Discovery**
   - Example from North:
   - Example from South:
   - Example from East/West:

💡 **Mind Blower** (1 advanced concept made fun)
   - "Scientists discovered..."
   - "In space..." 🌌

Rules:
1. Use only {class_num} grade NCERT vocabulary
2. Every 2 lines max. 8 words
3. 1 emoji every 10 words max
4. Include 3 comparison markers: "Like...", "Similar to...", "Just as..."
5. End with "What would happen if..." question

Text Content:
{text}"""

class APIKeyManager:
    STATE_FILE = "/content/drive/MyDrive/api_key_states.json"

    def __init__(self, api_keys):
        self.api_keys = api_keys
        self.blocked_keys = {}  # {key: block_until_timestamp}
        self.load_states()

    def load_states(self):
        try:
            if os.path.exists(self.STATE_FILE):
                with open(self.STATE_FILE, 'r') as f:
                    self.blocked_keys = json.load(f)
                    # Remove expired entries
                    now = time.time()
                    self.blocked_keys = {k: v for k, v in self.blocked_keys.items() if v > now}
        except Exception as e:
            print(f"Error loading key states: {e}")
            self.blocked_keys = {}

    def save_states(self):
        try:
            with open(self.STATE_FILE, 'w') as f:
                json.dump(self.blocked_keys, f)
        except Exception as e:
            print(f"Error saving key states: {e}")

    def get_available_keys(self):
        now = time.time()
        available = []
        for key in self.api_keys:
            if key not in self.blocked_keys or self.blocked_keys[key] < now:
                available.append(key)
        return available

    def block_key(self, key, hours=6):
        self.blocked_keys[key] = time.time() + (hours * 3600)
        self.save_states()
        print(f"🔒 Blocked key ending ...{key[-4:]} for {hours} hours")

def process_files_parallel(files, api_keys):
    key_manager = APIKeyManager(api_keys)
    file_queue = Queue()
    for f in files:
        file_queue.put(f)

    processed_lock = Lock()
    processed_count = 0
    total_files = len(files)

    def worker():
        nonlocal processed_count
        while not file_queue.empty():
            pdf_path = file_queue.get()

            available_keys = key_manager.get_available_keys()
            if not available_keys:
                print("⚠️ No available API keys. Waiting...")
                time.sleep(60)
                file_queue.put(pdf_path)
                continue

            # Randomize key selection
            key = random.choice(available_keys)
            ai = BharatShikshaAI(key)

            try:
                print(f"\n📄 Processing with key ...{key[-4:]}: {Path(pdf_path).name}")
                success = ai.process_pdf(pdf_path)

                with processed_lock:
                    processed_count += 1
                    print(f"📊 Progress: {processed_count}/{total_files} "
                          f"({(processed_count/total_files)*100:.1f}%)")

            except Exception as e:
                error_msg = str(e).lower()
                if 'rate_limit' in error_msg or 'quota' in error_msg:
                    key_manager.block_key(key)
                    file_queue.put(pdf_path)
                elif 'retries_exhausted' in error_msg:
                    key_manager.block_key(key, hours=0.5)  # Shorter block for temp errors
                    file_queue.put(pdf_path)
                else:
                    print(f"🔴 Unhandled error: {e}")
                    file_queue.put(pdf_path)

            file_queue.task_done()
            time.sleep(DELAY_SECONDS)

    # Create worker threads
    num_workers = min(len(api_keys), 8)  # Max 8 threads
    threads = []
    for _ in range(num_workers):
        t = Thread(target=worker)
        t.start()
        threads.append(t)

    # Wait for completion
    file_queue.join()

    # Stop workers
    for t in threads:
        t.join()

    print(f"✅ Completed processing {processed_count}/{total_files} files")
    print(f"🔑 Key usage report:")
    for key in api_keys:
        status = "blocked" if key in key_manager.blocked_keys else "active"
        print(f"• ...{key[-4:]} - {status}")

class BharatShikshaAI:
    def __init__(self, api_key):
        self.processed_files = set()
        self.api_key = api_key
        self.model = genai.GenerativeModel('models/gemini-1.5-pro-001',
                                      system_instruction="ALWAYS prioritize completeness over brevity. Include 100% of laws, formulas, diagrams, and examples from the source material.")
        self.client = google_genai.Client(
            api_key=self.api_key,
        )
        self.model_name = "gemini-2.0-pro-exp-02-05"
        self.quality_checks = [
            "Contains 5+ pedagogical elements",
            "Uses 3+ India-specific examples",
            "Matches NCERT learning outcomes",
            "Age-appropriate language",
            "100% factual accuracy"
        ]
        self.age_groups = {
            9: "13-14", 10: "14-15",
            11: "16-17", 12:
            "17-18"
        }
        self.experiment_samples = EXPERIMENT_TEMPLATE["sample"]
        self.cultural_regions = CULTURAL_EXAMPLES
        self.advanced_concepts = ADVANCED_CONCEPT_MAP

    def _format_error(self, error_type, message, solution=None, context=None):
        error_msg = f"\n🔴 [{time.ctime()}] ERROR ({error_type}): {message}"
        if context:
            error_msg += f"\n    Context: {context}"
        if solution:
            error_msg += f"\n    Suggested Fix: {solution}"
        return error_msg

    def process_file_chunk(self, files):
        """Process a subset of files using this instance's API key"""
        print(f"🚩 Starting processing chunk of {len(files)} files with key ending: {self.api_key[-4:]}")
        files_processed = 0

        for pdf_path in files:
            try:
                print(f"\n📄 Processing [{files_processed+1}/{len(files)}]: {Path(pdf_path).name}")
                success = self.process_pdf(pdf_path)
                if success:
                    files_processed += 1
                time.sleep(DELAY_SECONDS)  # Maintain delay between requests per key
            except Exception as e:
                print(f"🔴 Failed to process {Path(pdf_path).name}: {str(e)}")

        print(f"📊 Key {self.api_key[-4:]} processed {files_processed}/{len(files)} files")

    def get_allowed_languages(self, subject):
        """Determine permitted languages based on subject type"""
        subject_lower = subject.lower()

        # 2. Define core language subjects and their allowed languages
        LANGUAGE_SUBJECTS = {
            'sanskrit': ['sa', 'hi']
            'hindi': ['hi'],
            'english': ['en'],
            'urdu': ['ur'],       # Example additional language
            'telugu': ['te'],      # Example regional language
            'marathi': ['mr']      # Example regional language
        }

        # 3. Check if it's a language subject (excluding Sanskrit)
        for lang_subject, langs in LANGUAGE_SUBJECTS.items():
            if lang_subject == subject_lower:
                return langs

        # 4. For all non-language subjects (Science, Math, etc.)
        return ['en', 'hi', 'hinglish']

    def __del__(self):
        """Cleanup GRPC channels on destruction"""
        try:
            genai.global_shutdown()
        except Exception as e:
            pass

    def cleanup(self):
        """Graceful shutdown procedure"""
        print("\n🧹 Cleaning up resources...")
        genai.global_shutdown()
        time.sleep(2)

    def process_file(self, file_path):
        if not file_path.lower().endswith(".pdf"):
            print("❌ Only PDF files supported")
            return

        print(f"\n📄 Processing: {Path(file_path).name}")
        try:
            text = self.extract_text(file_path)
            if not text:
                return

            meta = self.extract_metadata(file_path)
            print(f"🏫 Class {meta['class']} | {meta['subject']} Chapter {meta['chapter']}")

            # Generate child-friendly summary
            child_summary = self.generate_child_friendly_summary(text, meta, "en")
            if child_summary:
                self.save_summary(child_summary, meta, "en", "child_friendly")

        except Exception as e:
            print(f"🔴 Error: {str(e)}")

    def process_problematic_files(self):
        if not PROBLEMATIC_FILES:
            return

        print("\n\n🔧 Problematic Files Report:")
        print("----------------------------")
        for file in PROBLEMATIC_FILES:
            print(f"File: {Path(file['path']).name}")
            print(f"Detected Class: {file['detected_class']}")
            print(f"Detected Subject: {file['detected_subject']}")
            print(f"Issue: {'Chapter number' if file['reason'] == 'chapter' else 'Subject'} detection failed")
            print("----------------------------")

        print("\n❓ Please provide missing details (or press Enter to skip):")
        print("Format: filename:chapter_number,subject")
        print("Example: ihsh101.pdf:01,Sanskrit\n")

        corrections = {}
        while True:
            user_input = input("Enter correction (or 'done' to finish): ").strip()
            if user_input.lower() in ['done', '']:
                break

            try:
                filename, details = user_input.split(":")
                chapter, subject = details.split(",")
                corrections[filename] = {
                    'chapter': f"{int(chapter):02d}",
                    'subject': subject.strip().title()
                }
            except ValueError:
                print("⚠️ Invalid format. Use: filename:chapter,subject")

        # Process corrected files
        for file in PROBLEMATIC_FILES[:]:
            filename = Path(file['path']).name
            if filename in corrections:
                print(f"\n🔨 Processing corrected file: {filename}")
                # Create modified metadata
                corrected_meta = {
                    'class': file['detected_class'],
                    'chapter': corrections[filename]['chapter'],
                    'subject': corrections[filename]['subject'],
                    'path': file['path'],
                    'valid': True
                }

                try:
                    text = self.extract_text(file['path'])
                    if text:
                        # Process all language versions
                        for lang_code in self.get_allowed_languages(corrected_meta['subject']):
                            for version in range(1, SUMMARY_VERSIONS + 1):
                                summary = self.generate_summary(text, corrected_meta, lang_code)
                                if summary:
                                    self.save_summary(summary, corrected_meta, lang_code, version)
                        PROBLEMATIC_FILES.remove(file)
                except Exception as e:
                    print(f"Failed to process corrected file {filename}: {str(e)}")


    def should_process(self, meta):
        """Check if summaries already exist for this file"""
        # Build expected output directory structure
        output_dir = Path(OUTPUT_DIR) / f"Class_{meta['class']}" / meta["subject"].replace(" ", "_") / meta["book"].replace(" ", "_") / f"Chapter_{meta['chapter']}"

        # Check if any markdown files exist in any language subdirectories
        if output_dir.exists():
            for lang_dir in output_dir.iterdir():
                if lang_dir.is_dir() and any(lang_dir.glob("*.md")):
                    return False

        # Check force regenerate conditions
        force_conditions = [
            meta["file_name"] in FORCE_REGENERATE,
            meta["book"] in FORCE_REGENERATE,
            str(Path(meta["path"]).as_posix() in FORCE_REGENERATE)
        ]

        return not output_dir.exists() or any(force_conditions)


    def get_existing_versions(self, meta, lang_code):
        """Count existing summary versions for a specific language"""
        version_pattern = re.compile(
            rf"^{re.escape(meta['subject'].replace(' ', '_'))}_"
            rf"{re.escape(meta['book'].replace(' ', '_'))}_"
            rf"{re.escape(meta['chapter'])}_"
            rf"{re.escape(lang_code)}_v(\d+)_"
            rf".+\.md$"
        )

        output_dir = Path(OUTPUT_DIR) / f"Class_{meta['class']}" / meta["subject"].replace(" ", "_") / meta["book"].replace(" ", "_") / f"Chapter_{meta['chapter']}" / lang_code

        if not output_dir.exists():
            return 0

        return len([
            f for f in os.listdir(output_dir)
            if version_pattern.match(f)
        ])

    def should_process_lang(self, meta, lang_code):
        """Check if a language needs more versions"""
        if any([
            meta["file_name"] in FORCE_REGENERATE,
            meta["book"] in FORCE_REGENERATE,
            str(Path(meta["path"]).as_posix() in FORCE_REGENERATE)
        ]):
            return SUMMARY_VERSIONS  # Force full regeneration

        existing = self.get_existing_versions(meta, lang_code)
        return max(SUMMARY_VERSIONS - existing, 0)


    def process_pdf(self, pdf_path):
        try:
            meta = self.extract_metadata(pdf_path)
            if not meta['valid']:
                print(f"⏭ Skipping file due to invalid metadata: {Path(pdf_path).name}")
                return False

            print(f"📄 Metadata extracted: {meta}")

            text = self.extract_text(pdf_path)
            if not text:
                print("🚫 No text extracted")
                return False

            # Validate minimum text length
            if len(text.strip()) < 1000:
                print(f"⚠️ Insufficient text ({len(text)} chars), skipping")
                return False

            print(f"📖 Text length: {len(text)} characters")

            allowed_langs = self.get_allowed_languages(meta["subject"])
            print(f"🌐 Allowed languages: {allowed_langs}")

            success = False
            for lang_code in allowed_langs:
                print(f"\n🔵 Processing language: {lang_code}")

                # Get existing version count
                existing_versions = self.get_existing_versions(meta, lang_code)
                versions_needed = max(SUMMARY_VERSIONS - existing_versions, 0)

                if versions_needed <= 0:
                    print(f"⏩ Skipping {lang_code} - has all {SUMMARY_VERSIONS} versions")
                    continue

                # Generate needed versions
                for version_offset in range(versions_needed):
                    attempt_number = version_offset + 1
                    print(f"🔄 Attempting version {existing_versions + attempt_number}/{SUMMARY_VERSIONS}")

                    summary = self.generate_summary(text, meta, lang_code)
                    if summary:
                        self.save_summary(summary, meta, lang_code, 0)
                        success = True
                    else:
                        print(f"❌ Failed to generate version {existing_versions + attempt_number}")

                    time.sleep(DELAY_SECONDS)

                # Child-friendly version (English only)
                if lang_code == "en":
                    cf_path = self.get_child_friendly_path(meta)
                    if not cf_path.exists() or any([
                        meta["file_name"] in FORCE_REGENERATE,
                        meta["book"] in FORCE_REGENERATE
                    ]):
                        print("🎨 Attempting child-friendly version")
                        child_summary = self.generate_child_friendly_summary(text, meta, lang_code)
                        if child_summary:
                            print("🧒 Child summary generated")
                            self.save_summary(child_summary, meta, lang_code, "child_friendly")
                            success = True
                        else:
                            print("👶 Failed child-friendly summary")
                    else:
                        print(f"⏩ Skipping child-friendly - exists at {cf_path}")

            return success

        except KeyError as ke:
            error_msg = self._format_error(
                "CONFIG_MISSING",
                f"Missing configuration key: {str(ke)}",
                solution="Update ADVANCED_CONCEPT_MAP or CULTURAL_EXAMPLES",
                context={
                    "file": pdf_path,
                    "missing_key": str(ke)
                }
            )
            print(error_msg)
            return False
        except Exception as e:
            error_msg = self._format_error(
                "PROCESSING_FAILURE",
                f"Critical processing failure: {str(e)}",
                solution="Check file content and API availability",
                context={
                    "file": pdf_path,
                    "stack_trace": traceback.format_exc()
                }
            )
            print(error_msg)
            return False

    def extract_text(self, path):
        try:
            text = ""
            with open(path, "rb") as f:
                reader = PyPDF2.PdfReader(f)

                if reader.is_encrypted:
                    try:
                        reader.decrypt('')  # Try empty password
                    except Exception as e:
                        print(f"🔒 Decryption failed: {str(e)}")
                        return None

                for page in reader.pages:
                    try:
                        page_text = page.extract_text()
                        if page_text:
                            # Clean text artifacts
                            page_text = re.sub(r'\s+-\s+', '-', page_text)  # Fix hyphenation
                            page_text = re.sub(r'(?<!\n)\n(?!\n)', ' ', page_text)  # Join broken lines
                            text += page_text + "\n"
                    except Exception as page_error:
                        print(f"📄 Page error: {str(page_error)}")
                        continue

            return text if len(text.strip()) > 500 else None  # Increased minimum threshold
        except Exception as e:
            print(f"❓ Extraction error: {str(e)}")
            return None

    def extract_metadata(self, path):
        try:
            path_obj = Path(path)
            parts = path_obj.parts
            file_name = path_obj.stem
            original_file_name = path_obj.name

            # Get directory-based book name if single PDF in folder
            parent_dir = path_obj.parent
            pdfs_in_dir = [f for f in os.listdir(parent_dir) if f.lower().endswith(".pdf")]

            book_name = parent_dir.name if len(pdfs_in_dir) == 1 else file_name

            # Debug: Print path parts
            print(f"🔍 Path parts: {parts}")

            # Default values
            class_num = 0
            subject = "General"
            book_name = "General"
            chapter_num = "00"

            # 1. Class Detection
            class_num = 0  # default
            for part in parts:
                if "class" in part.lower():
                    class_match = re.search(r'\d+', part)
                    if class_match:
                        class_num = min(max(int(class_match.group()), 1), 12)
                        break

            # 2. Subject Detection from Directory Structure
            subject = "Unknown"
            try:
                class_index = parts.index(f"Class_{class_num}")
                if len(parts) > class_index + 1:
                    # Get first directory after class folder as subject
                    subject = parts[class_index + 1].replace('_', ' ').title()
            except (ValueError, IndexError):
                pass

            VALID_SUBJECTS = {
                'Maths', 'Science', 'Hindi', 'English',
                'Sanskrit', 'Social_Studies', 'ICT', 'Physics',
                'Chemistry', 'Biology', 'History', 'Geography', 'physical_education'
            }
            subject_valid = subject in VALID_SUBJECTS

            # Extract book name from directory after subject
            try:
                # Get the directory immediately after subject
                subject_dir = path_obj.parent
                if subject_dir.name == subject.replace(' ', '_'):
                    # If in subject root directory
                    book_name = "General"
                else:
                    # Use first subdirectory after subject as book name
                    book_name = subject_dir.name.replace('_', ' ').title()
            except Exception:
                book_name = "Unknown"

            chapter_match = re.search(r'(\d{1,3})(?=\.pdf$)', original_file_name, re.IGNORECASE)
            if chapter_match:
                chapter_num = chapter_match.group(1).zfill(2)  # Pad with leading zero
                print(f"✅ Chapter number found: {chapter_num}")
                chapter_valid = True
            else:
                print(f"⚠️ No chapter number found in {original_file_name}")
                chapter_valid = False

            # Validate subject detection
            subject_valid = subject != "Unknown"

            return {
            "class": min(max(class_num, 1), 12),
            "chapter": chapter_num,
            "subject": subject.replace('_', ' ').title(),
            "book": book_name.replace('_', ' ').title(),
            "path": str(path),
            "file_name": file_name,
            "valid": chapter_valid and subject_valid
            }

            if not metadata['valid']:
                print(f"🚨 Metadata validation failed for {original_file_name}")
                PROBLEMATIC_FILES.append({
                    'path': path,
                    'detected_class': metadata['class'],
                    'detected_subject': metadata['subject'],
                    'reason': 'chapter' if not chapter_valid else 'subject'
                })

            return metadata
        except Exception as e:
            error_msg = self._format_error(
                "METADATA_EXTRACTION_FAILURE",
                f"Could not extract metadata: {str(e)}",
                solution="Check file naming convention and directory structure",
                context={
                    "path": path,
                    "error_location": "Chapter number detection" if "chapter" in str(e)
                                  else "Subject/class detection"
                }
            )
            print(error_msg)
            return {
                "class": 9,
                "chapter": "01",
                "subject": "General",
                "book": "General",
                "path": str(path),
                "file_name": "unknown"
            }

    def generate_summary(self, text, meta, lang_code):
        for attempt in range(MAX_RETRIES + 1):
            try:
                prompt = self.build_prompt(text, meta, lang_code)
                contents = [
                    types.Content(
                        role="user",
                        parts=[types.Part.from_text(text=prompt)],
                    ),
                ]

                config = types.GenerateContentConfig(
                    temperature=0.3,
                    top_p=0.95,
                    top_k=64,
                    max_output_tokens=self.get_token_limit(meta["class"]),
                    response_mime_type="text/plain",
                )

                full_response = []
                for chunk in self.client.models.generate_content_stream(
                    model=self.model_name,
                    contents=contents,
                    config=config,
                ):
                    if chunk.text:
                        full_response.append(chunk.text)
                        print(chunk.text, end="")  # Stream output

                final_response = "".join(full_response)
                if self.validate_summary(final_response, meta["class"]):
                    return final_response

            except Exception as e:
                error_details = str(e)
                error_context = {
                    "file": meta["path"],
                    "class": meta["class"],
                    "chapter": meta["chapter"],
                    "language": lang_code,
                    "attempt": attempt+1
                }

                error_type = type(e).__name__
                error_details = str(e)

                if "429" in error_details:
                    error_type = "RATE_LIMIT"
                    error_msg = self._format_error(
                        "RATE_LIMIT",
                        "API request limit exceeded",
                        solution=f"Increase DELAY_SECONDS (current: {DELAY_SECONDS}s) or add more API keys",
                        context=error_context
                    )
                elif "500" in error_details:
                    error_type = "SERVER_ERROR"
                    error_msg = self._format_error(
                        "SERVER_ERROR",
                        "Google API server issues",
                        solution="Retry after some time",
                        context=error_context
                    )
                else:
                    error_type = "GENERATION_ERROR"
                    error_msg = self._format_error(
                        "GENERATION_ERROR",
                        f"Failed to generate content ({error_details})",
                        solution="Check prompt formatting and API key permissions",
                        context=error_context
                    )

                print(error_msg)
                raise Exception(f"{error_type}: {error_details}") from e

                jitter = random.uniform(0.5, 1.5)
                wait_time = (2 ** attempt) * 3 * jitter
                print(f"⏳ Waiting {wait_time:.1f}s before retry {attempt+1}/{MAX_RETRIES}")
                time.sleep(wait_time)

        raise Exception("MAX_RETRIES_EXHAUSTED: Failed after {MAX_RETRIES} attempts")

        print(self._format_error(
            "MAX_RETRIES_EXHAUSTED",
            f"Failed after {MAX_RETRIES} attempts",
            solution="Check API keys or reduce concurrent requests",
            context={
                "file": meta["path"],
                "last_error": error_details
            }
        ))
        return None

    def get_education_level(self, class_num):
        """Categorize class into education levels"""
        if class_num <= 5:
            return "primary"
        elif class_num <= 8:
            return "middle"
        else:  # 9-12
            return "secondary"

    def generate_child_friendly_summary(self, text, meta, lang_code):
        prompt = self.build_child_prompt(text, meta, lang_code)
        try:
            # Use the configured model instance
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"Child summary error: {str(e)}")
            return None

    def build_prompt(self, text, meta, lang_code):
        class_num = meta["class"]
        return PEDAGOGICAL_PROMPT.format(
            class_num=class_num,
            chapter_title = f"{meta['book']} - Chapter {meta['chapter']}",
            subject=meta["subject"],
            chapter_num=meta["chapter"],
            lang_name=SUPPORTED_LANGS[lang_code],
            lang_code=lang_code,
            state_board="State_Board" if class_num < 10 else "NCERT",
            cultural_ref=self.get_cultural_reference(class_num),
            concept_style=self.get_concept_style(class_num),
            learning_points=self.get_learning_points(class_num),
            activity=self.get_activity(class_num),
            discussion_topic=self.get_discussion_topic(class_num),
            assessment_types=self.get_assessment_types(class_num),
            indian_examples=self.get_indian_examples(class_num),
            blooms_level=self.get_blooms_level(class_num),
            lang_rules=self.get_lang_rules(lang_code),
            quality_checks=", ".join(self.quality_checks),
            token_limit=self.get_token_limit(class_num),
            text=text
        )

    def build_child_prompt(self, text, meta, lang_code):
        education_level = self.get_education_level(meta["class"])
        experiments = self.experiment_samples.get(meta["subject"], ["Simple observation exercise"]*3)
        try:
            adv_concept = next(iter(self.advanced_concepts[education_level].keys()))
            adv_explanation = next(iter(self.advanced_concepts[education_level].values()))
        except (KeyError, StopIteration):
            adv_concept = "Space Exploration"
            adv_explanation = "Rockets that reach Mars 🚀🪐"

        return CHILD_FRIENDLY_PROMPT_V2.format(
            class_num=meta["class"],
            chapter_title=f"Chapter {meta['chapter']}",
            lang_name=SUPPORTED_LANGS[lang_code],
            age_group=self.age_groups.get(meta["class"], "10-11"),
            experiment1=experiments[0],
            experiment2=experiments[1],
            experiment3=experiments[2],
            north_example=self.cultural_regions["north"][meta["class"] % 3],
            south_example=self.cultural_regions["south"][meta["class"] % 3],
            east_west_example=self.cultural_regions["east"][meta["class"] % 3],
            adv_concept=next(iter(self.advanced_concepts[education_level].keys())),
            adv_explanation=next(iter(self.advanced_concepts[education_level].values())),
            text=text[:10000]
        )

    def validate_summary(self, content, class_num):
        # Temporary override
        print("⚠️ Validation bypassed for debugging")
        return True  # Force accept all content

    def save_summary(self, content, meta, lang_code, version_type):
        try:
            # Determine if we need to force regenerate
            force = any([
                meta["file_name"] in FORCE_REGENERATE,
                meta["book"] in FORCE_REGENERATE,
                str(Path(meta["path"]).parent.as_posix()) in FORCE_REGENERATE
            ])

            # Clean and format filename components
            subject_clean = meta["subject"].replace(" ", "_")
            book_clean = meta["book"].replace(" ", "_")
            chapter_clean = meta["chapter"].zfill(2)
            original_file = Path(meta["path"]).stem

            output_path = os.path.join(
                OUTPUT_DIR,
                f"Class_{meta['class']}",
                subject_clean,
                book_clean,
                f"Chapter_{chapter_clean}",
                lang_code
            )

            # Handle version type
            if version_type == "child_friendly":
                filename = (
                    f"{subject_clean}_"
                    f"{book_clean}_"
                    f"{chapter_clean}_"
                    f"{lang_code}_"
                    f"child-friendly_"
                    f"{original_file}_Summary.md"
                )
                full_path = os.path.join(output_path, filename)

                # Skip if exists unless forced
                if not force and os.path.exists(full_path):
                    return
            else:
                # Find next available version for regular summaries
                new_version = 1
                while True:
                    test_filename = (
                        f"{subject_clean}_"
                        f"{book_clean}_"
                        f"{chapter_clean}_"
                        f"{lang_code}_"
                        f"v{new_version}_"
                        f"{original_file}_Summary.md"
                    )
                    test_path = os.path.join(output_path, test_filename)

                    if not os.path.exists(test_path) or (force and new_version == 1):
                        break
                    new_version += 1

                # Enforce version limit
                if new_version > SUMMARY_VERSIONS:
                    print(f"⚠️ Version {new_version} exceeds max allowed ({SUMMARY_VERSIONS}), skipping.")
                    return

                filename = (
                    f"{subject_clean}_"
                    f"{book_clean}_"
                    f"{chapter_clean}_"
                    f"{lang_code}_"
                    f"v{new_version}_"
                    f"{original_file}_Summary.md"
                )
                full_path = os.path.join(output_path, filename)


            # Skip existing identical content unless forced
            if not force and os.path.exists(full_path):
                with open(full_path, "r", encoding="utf-8") as f:
                    if f.read() == content:
                        print(f"⏩ Skipping existing version v{new_version}")
                        return

            # Create directory structure
            Path(output_path).mkdir(parents=True, exist_ok=True)

            # Clean up any existing file if forcing
            if force and os.path.exists(full_path):
                os.remove(full_path)

            # Write content
            with open(full_path, "w", encoding="utf-8") as f:
                f.write(f"# Class {meta['class']} {meta['subject']} - {meta['book']} Chapter {meta['chapter']}\n")
                f.write(f"**Language:** {SUPPORTED_LANGS[lang_code]}\n\n")
                f.write(content)

            print(f"💾 Saved to: {full_path} (v{new_version}/{SUMMARY_VERSIONS})")

        except PermissionError as pe:
            error_msg = self._format_error(
                "PERMISSION_DENIED",
                f"Cannot write to {output_path}",
                solution="Check Google Drive write permissions",
                context={
                    "path": output_path,
                    "os_user": os.getlogin()
                }
            )
            print(error_msg)
        except FileNotFoundError as fnfe:
            error_msg = self._format_error(
                "PATH_INVALID",
                f"Invalid directory structure: {fnfe}",
                solution="Verify OUTPUT_DIR configuration",
                context={
                    "attempted_path": output_path,
                    "meta_data": meta
                }
            )
            print(error_msg)
        except Exception as e:
            error_msg = self._format_error(
                "SAVE_FAILURE",
                f"Unexpected error saving summary: {str(e)}",
                solution="Check disk space and file permissions",
                context={
                    "file": meta["path"],
                    "output_path": full_path
                }
            )
            print(error_msg)

    def get_lang_rules(self, lang_code):
        return {
            "en": "Formal English with CBSE terms",
            "hi": "Simple Hindi with NCERT terms",
            "hinglish": "60% Hindi/40% English mix",
            "sa": "सरल संस्कृतम् with शुद्धपाठः"
        }[lang_code]

    # Pedagogical content generators remain same
    def get_cultural_reference(self, class_num):
        return ["Toys/Festivals", "Historical Sites", "Economic Data"][
            0 if class_num<6 else 1 if class_num<9 else 2]

    def get_concept_style(self, class_num):
        return {
            "primary": "3 emoji-annotated points 🎯",
            "middle": "Flowcharts with examples ➡️",
            "secondary": "Detailed concept hierarchy 📊"
        }[self.get_education_level(class_num)]

    def get_learning_points(self, class_num):
        return {
            "primary": "Simple analogies with drawings ✍️",
            "middle": "5 bullet points with NCERT references 📚",
            "secondary": "Detailed explanations with diagrams 📈"
        }[self.get_education_level(class_num)]

    def get_activity(self, class_num):
        return {
            "primary": "Hands-on craft with local materials 🎨",
            "middle": "Group project with presentations 👥",
            "secondary": "Research-based case study analysis 🔍"
        }[self.get_education_level(class_num)]

    def get_discussion_topic(self, class_num):
        return {
            "primary": "Why is this important in our daily life? 🤔",
            "middle": "How does this affect our community? 🏘️",
            "secondary": "Critical analysis of real-world impacts 🌍"
        }[self.get_education_level(class_num)]

    def get_assessment_types(self, class_num):
        return {
            "primary": "Picture-based questions 🖼️",
            "middle": "3-mark short answers ✍️",
            "secondary": "Case studies & diagrams 📝"
        }[self.get_education_level(class_num)]

    def get_indian_examples(self, class_num):
        return {
            "primary": "Local festivals and traditions 🪔",
            "middle": "State-specific case studies 📌",
            "secondary": "National economic/social data 📊"
        }[self.get_education_level(class_num)]

    def get_blooms_level(self, class_num):
        return {
            "primary": "Remembering/Understanding",
            "middle": "Applying/Analyzing",
            "secondary": "Evaluating/Creating"
        }[self.get_education_level(class_num)]

    def get_token_limit(self, class_num):
        """Maximum output tokens for detailed summaries"""
        return 8192  # Max for Gemini 1.5 Pro

if __name__ == "__main__":
    print("🚀 Starting Bharat Shiksha AI System")

    # Collect all PDF files
    all_pdfs = []
    for root, dirs, files in os.walk(INPUT_DIR):
        all_pdfs.extend(os.path.join(root, f) for f in files if f.lower().endswith(".pdf"))

    print(f"📦 Total files to process: {len(all_pdfs)}")

    # Initial processing
    process_files_parallel(all_pdfs, API_KEYS)

    # Handle problematic files
    if PROBLEMATIC_FILES:
        ai = BharatShikshaAI(API_KEYS[0])  # Use first API key for corrections
        ai.process_problematic_files()

        if PROBLEMATIC_FILES:
            print("\n⚠️ Remaining unresolved files:")
            for file in PROBLEMATIC_FILES:
                print(f"- {Path(file['path']).name}")
            print("Please check the file naming convention and rerun the script.")

    print("✅ All processing completed")

Streaming output truncated to the last 5000 lines.
🔵 Processing language: en
⏩ Skipping en - has all 3 versions

🔵 Processing language: hi
⏩ Skipping hi - has all 3 versions

🔵 Processing language: hinglish
⏩ Skipping hinglish - has all 3 versions
📊 Progress: 57/157 (36.3%)
📖 Text length: 34366 characters
🌐 Allowed languages: ['en', 'hi', 'hinglish']

🔵 Processing language: en
⏩ Skipping en - has all 3 versions

🔵 Processing language: hi
⏩ Skipping hi - has all 3 versions

🔵 Processing language: hinglish
⏩ Skipping hinglish - has all 3 versions
📊 Progress: 58/157 (36.9%)

📄 Processing with key ...AzTI: iehp103.pdf
🔍 Path parts: ('/', 'content', 'drive', 'MyDrive', 'Schools_Books', 'Class_9', 'physical_education', 'iehp103.pdf')
✅ Chapter number found: 103
📄 Metadata extracted: {'class': 9, 'chapter': '103', 'subject': 'Physical Education', 'book': 'Physical Education', 'path': '/content/drive/MyDrive/Schools_Books/Class_9/physical_education/iehp103.pdf', 'file_name': 'iehp103', 'valid'

In [ ]:
import os
import re
from collections import defaultdict
from pathlib import Path
from google.colab import drive

SUPPORTED_LANGS = {"en", "hi", "sa", "hinglish"}

class SummaryAnalyzer:
    def __init__(self, summary_root="/content/drive/MyDrive/Schools_Books_Summaries"):
        self.summary_root = summary_root
        self.analysis = defaultdict(lambda: {
            'subjects': defaultdict(lambda: {
                'books': defaultdict(lambda: {
                    'chapters': defaultdict(lambda: {
                        'languages': defaultdict(lambda: {
                            'versions': defaultdict(int)
                        })
                    })
                })
            })
        })
        self.problem_files = []

    def _parse_filename(self, filename):
        """Improved filename parser that handles complex names"""
        try:
            if not filename.endswith("_Summary.md"):
                raise ValueError("Invalid filename format")

            # Remove suffix and split into parts
            clean_name = filename[:-11]  # Remove "_Summary.md"
            parts = clean_name.split('_')

            # Need at least: [subject/book][chapter][lang][version][original]
            if len(parts) < 5:
                raise ValueError("Insufficient filename components")

            # Extract from end backwards
            original_file = parts[-1]
            version = parts[-2][1:] if parts[-2].startswith("v") else parts[-2]
            language = parts[-3]
            chapter = parts[-4]

            # Validate core fields
            if language not in SUPPORTED_LANGS:
                raise ValueError(f"Unsupported language: {language}")
            if not chapter.isdigit():
                raise ValueError(f"Invalid chapter: {chapter}")

            # Split subject/book (remaining parts)
            subject_book = parts[:-4]
            if len(subject_book) < 2:
                raise ValueError("Missing subject/book information")

            return {
                'subject': '_'.join(subject_book[:-1]),
                'book': subject_book[-1],
                'chapter': chapter,
                'language': language,
                'version': version,
                'original_file': original_file
            }

        except Exception as e:
            self.problem_files.append(f"{filename} ({str(e)})")
            return None

    def _parse_path(self, path):
        """Parse class from directory path"""
        try:
            class_part = [p for p in Path(path).parts if p.startswith('Class_')][0]
            class_num = int(re.search(r'Class_(\d+)', class_part).group(1))
            if 1 <= class_num <= 12:
                return class_num
            raise ValueError("Invalid class number")
        except Exception as e:
            self.problem_files.append(f"{path} ({str(e)})")
            return None

    def analyze(self):
        """Process files and generate analysis"""
        for root, dirs, files in os.walk(self.summary_root):
            for file in files:
                if file.endswith(".md"):
                    filepath = os.path.join(root, file)
                    class_num = self._parse_path(root)
                    meta = self._parse_filename(file)

                    if not meta or not class_num:
                        continue

                    # Update analysis structure
                    class_data = self.analysis[class_num]
                    subject_data = class_data['subjects'][meta['subject']]
                    book_data = subject_data['books'][meta['book']]
                    chapter_data = book_data['chapters'][int(meta['chapter'])]
                    lang_data = chapter_data['languages'][meta['language']]
                    lang_data['versions'][meta['version']] += 1

        return self.generate_report()

    def generate_report(self):
        """Generate structured hierarchical report"""
        report = ["\n📚 Hierarchical Summary Report", "==========================="]

        # Class-level reporting
        for class_num in sorted(self.analysis.keys()):
            class_report = [f"\n🏫 Class {class_num}:"]
            class_data = self.analysis[class_num]

            # Subject-level
            for subject in sorted(class_data['subjects'].keys()):
                subject_report = [f"\n  📖 Subject: {subject.replace('_', ' ')}"]
                subject_data = class_data['subjects'][subject]

                # Book-level
                for book in sorted(subject_data['books'].keys()):
                    book_report = [f"\n    📗 Book: {book.replace('_', ' ')}"]
                    book_data = subject_data['books'][book]

                    # Chapter-level
                    for chapter in sorted(book_data['chapters'].keys()):
                        chapter_report = [f"\n      📄 Chapter {chapter}:"]
                        chapter_data = book_data['chapters'][chapter]

                        # Language-level
                        for lang in sorted(chapter_data['languages'].keys()):
                            lang_data = chapter_data['languages'][lang]
                            versions = sum(lang_data['versions'].values())
                            unique_versions = len(lang_data['versions'])
                            lang_name = {
                                'en': 'English', 'hi': 'Hindi',
                                'sa': 'Sanskrit', 'hinglish': 'Hinglish'
                            }.get(lang, lang)
                            chapter_report.append(
                                f"        🌐 {lang_name} - {versions} summaries "
                                f"({unique_versions} versions)"
                            )

                        book_report.append('\n'.join(chapter_report))

                    subject_report.append('\n'.join(book_report))

                class_report.append('\n'.join(subject_report))

            report.append('\n'.join(class_report))

        # Add problem files section
        if self.problem_files:
            report.append("\n\n⚠️ Validation Issues:")
            report.append(f"Found {len(self.problem_files)} problematic entries")
            report.append("Sample issues:")
            for f in self.problem_files[:3]:
                report.append(f"  - {f}")
            report.append("(Full list available in analyzer.problem_files)")

        return '\n'.join(report)

# Usage
if __name__ == "__main__":
    drive.mount('/content/drive')
    analyzer = SummaryAnalyzer()
    print(analyzer.analyze())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

📚 Hierarchical Summary Report

🏫 Class 9:

  📖 Subject: English

    📗 Book: Beehive

      📄 Chapter 101:
        🌐 English - 3 summaries (3 versions)

      📄 Chapter 102:
        🌐 English - 3 summaries (3 versions)

      📄 Chapter 103:
        🌐 English - 3 summaries (3 versions)

      📄 Chapter 104:
        🌐 English - 3 summaries (3 versions)

      📄 Chapter 105:
        🌐 English - 3 summaries (3 versions)

      📄 Chapter 106:
        🌐 English - 3 summaries (3 versions)

      📄 Chapter 107:
        🌐 English - 3 summaries (3 versions)

      📄 Chapter 108:
        🌐 English - 3 summaries (3 versions)

      📄 Chapter 109:
        🌐 English - 3 summaries (3 versions)

  📖 Subject: English Moments

    📗 Book: Supplementry

      📄 Chapter 101:
        🌐 English - 3 summaries (3 versions)

      📄 Chapter 102:
        🌐 English - 3 summaries (3 ve